In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import inf
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report,plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [2]:
features = pd.read_csv("./features/featMat.csv", names = ["user ID", "doc ID", "inter-stroke time", "stroke duration",  "start x",  "start y", "stop x",  "stop y",  "direct end-to-end distance", " mean resultant length", "up/down/left/right flag", "direction of end-to-end line", "phone ID",  "20%-perc. pairwise velocity",  "50%-perc. pairwise velocity",  "80%-perc. pairwise velocity",  "20%-perc. pairwise acc",  "50%-perc. pairwise acc",  "80%-perc. pairwise acc",  "median velocity at last 3 pts",  "largest deviation from end-to-end line",  "20%-perc. dev. from end-to-end line",  "50%-perc. dev. from end-to-end line",  "80%-perc. dev. from end-to-end line",  "average direction",  "length of trajectory", "ratio end-to-end dist and length of trajectory",  "average velocity",  "median acceleration at first 5 points",  "mid-stroke pressure", "mid-stroke area covered",  "mid-stroke finger orientation",  "change of finger orientation", "phone orientation"])

In [3]:
features.replace([np.inf, -np.inf], np.nan)

,user ID,doc ID,inter-stroke time,stroke duration,start x,start y,stop x,stop y,direct end-to-end distance,mean resultant length,...,average direction,length of trajectory,ratio end-to-end dist and length of trajectory,average velocity,median acceleration at first 5 points,mid-stroke pressure,mid-stroke area covered,mid-stroke finger orientation,change of finger orientation,phone orientation
0,37,4,0.341,0.256,27.416,-27.113,12.3980,-32.960,16.1160,0.35736,...,-2.581200,16.1320,0.99898,63.0180,-432.24,0.320,0.044444,0.0,0,1
1,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,...,-2.199400,12.5960,0.99197,146.4700,5117.70,0.600,0.088889,0.0,0,1
2,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,...,0.086288,16.2030,0.98959,145.9700,-3563.50,0.480,0.044444,0.0,0,1
3,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,...,-2.723100,6.9674,0.97902,74.1220,-4604.10,0.400,0.044444,0.0,0,1
4,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,...,-2.717000,13.8710,0.99965,247.6900,-21377.00,0.440,0.044444,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21153,13,2,4.837,2.357,26.811,-39.410,31.9520,-20.562,19.5370,0.74752,...,1.049300,23.2420,0.84058,9.8608,0.00,0.460,0.044444,0.0,0,1
21154,13,2,2.821,2.282,28.625,-37.092,31.5480,-19.756,17.5810,0.69342,...,0.666240,19.5190,0.90073,8.5534,656.21,0.420,0.044444,0.0,0,1
21155,13,2,3.038,0.204,28.121,-32.153,29.6330,-23.183,9.0972,0.90239,...,1.240200,9.3179,0.97631,45.6760,-2849.80,0.425,0.044444,0.0,0,1
21156,13,2,1.045,0.485,30.037,-34.774,30.1370,-27.315,7.4594,0.57243,...,0.986920,7.9983,0.93262,16.4910,0.00,0.360,0.044444,0.0,0,1


In [4]:
features.dropna()

,user ID,doc ID,inter-stroke time,stroke duration,start x,start y,stop x,stop y,direct end-to-end distance,mean resultant length,...,average direction,length of trajectory,ratio end-to-end dist and length of trajectory,average velocity,median acceleration at first 5 points,mid-stroke pressure,mid-stroke area covered,mid-stroke finger orientation,change of finger orientation,phone orientation
0,37,4,0.341,0.256,27.416,-27.113,12.3980,-32.960,16.1160,0.35736,...,-2.581200,16.1320,0.99898,63.0180,-432.24,0.320,0.044444,0.0,0,1
1,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,...,-2.199400,12.5960,0.99197,146.4700,5117.70,0.600,0.088889,0.0,0,1
2,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,...,0.086288,16.2030,0.98959,145.9700,-3563.50,0.480,0.044444,0.0,0,1
3,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,...,-2.723100,6.9674,0.97902,74.1220,-4604.10,0.400,0.044444,0.0,0,1
4,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,...,-2.717000,13.8710,0.99965,247.6900,-21377.00,0.440,0.044444,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21153,13,2,4.837,2.357,26.811,-39.410,31.9520,-20.562,19.5370,0.74752,...,1.049300,23.2420,0.84058,9.8608,0.00,0.460,0.044444,0.0,0,1
21154,13,2,2.821,2.282,28.625,-37.092,31.5480,-19.756,17.5810,0.69342,...,0.666240,19.5190,0.90073,8.5534,656.21,0.420,0.044444,0.0,0,1
21155,13,2,3.038,0.204,28.121,-32.153,29.6330,-23.183,9.0972,0.90239,...,1.240200,9.3179,0.97631,45.6760,-2849.80,0.425,0.044444,0.0,0,1
21156,13,2,1.045,0.485,30.037,-34.774,30.1370,-27.315,7.4594,0.57243,...,0.986920,7.9983,0.93262,16.4910,0.00,0.360,0.044444,0.0,0,1


In [1]:
# Q.2 Implement two more features in addition to the 30 found in the database. 
# Do they have positive information gain? That is, are the features useful?

# Hint: You may use PCA to generate the two features. 



In [ ]:
# Report correlation of these feature to the rest of the implemented features.
# Your code 

In [ ]:
'''
Train your model on a binary classifier of your choice (true user or false user classification problem)
using the following 4 scenarios in which you use a feature selection method to choose top 10 features.
Describe this process. Use 10-fold cross validation to compute precision and recall in the followingscenarios. 
Try to maximize F1 score when optimizing your classifier. 
Report F1 and any methodsyou used to optimize your classifier.
   (a) 10 top features;
   (b) 10 top features & your features
   
'''

# your code

# Question 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# Anomaly Detection
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score

raw_data = pd.read_csv('server-log.txt', sep=" ")
print(raw_data)


In [ ]:
# Identify the exact date and time1. What approach did the attackers use?


# Your code here




In [ ]:
# Identify the exact date and time1. What approach did the attackers use?


# Your explaination should go here

In [ ]:
#There has been significant literature discussing how entropy can be used to detect these attacks.
# To do it effectively, approximation schemes are usually used. You do not have to implement these
# approximation techniques, but do present an analysis of whether entropy is useful and which 
#com-binations you tried, e.g.src ip,dest ip,src-port,dst-port, etc. Do any reveal anomalies when 
#the two attacks happen?


# Your code here


In [ ]:
#There has been significant literature23discussing how entropy can be used to detect these attacks.
# To do it effectively, approximation schemes are usually used. You do not have to implement these
# approximation techniques, but do present an analysis of whether entropy is useful and which 
#com-binations you tried, e.g.src ip,dest ip,src-port,dst-port, etc. Do any reveal anomalies when 
#the two attacks happen?


# Your explaination should go here